In [1]:
import numpy as np
import pickle
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
train_imgs = pickle.load(open("train_images_512.pk",'rb'), encoding='bytes')
train_labels = pickle.load(open("train_labels_512.pk",'rb'), encoding='bytes')
test_imgs = pickle.load(open("test_images_512.pk",'rb'), encoding='bytes')    

In [3]:
base_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(512, 512, 3))
base_inception.trainable = False
feature_extractor = Model(inputs=base_inception.input, outputs=GlobalMaxPooling2D()(base_inception.output))

In [4]:
%%time
X_train = np.swapaxes(train_imgs.numpy(), 1, -1)
Z_train = feature_extractor.predict(X_train)
y_train = train_labels.numpy()

X_test = np.swapaxes(test_imgs.numpy(), 1, -1)
Z_test = feature_extractor.predict(X_test)

CPU times: user 1min 26s, sys: 11.4 s, total: 1min 38s
Wall time: 18.4 s


In [5]:
from sklearn.model_selection import GridSearchCV

def optimize(model, parameter_space):
    clf = GridSearchCV(model, parameter_space, scoring='f1', cv=5, n_jobs=-1)
    clf.fit(Z_train, y_train)
    return clf.best_estimator_, clf.best_score_, clf.best_params_

In [6]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

def optimizeSVC():
    parameter_space = {
    'kernel': ['rbf', 'linear', 'poly'],
    'C': [0.05, 1, 10, 50, 100, 200],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None],
    'degree': [2, 3, 4, 5, 6]
    }
    return optimize(SVC(probability=True), parameter_space)

def optimizeKNN():
    parameter_space = {
        'n_neighbors': [1, 2, 3, 4, 5],
        'weights': ['uniform', 'distance']
    }
    return optimize(KNeighborsClassifier(), parameter_space)

In [7]:
%%time
svc_model, svc_score, svc_params = optimizeSVC()
knn_model, knn_score, knn_params = optimizeKNN()

CPU times: user 2.43 s, sys: 275 ms, total: 2.71 s
Wall time: 13.5 s


In [8]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

eclf = VotingClassifier(estimators=[('svc', svc_model), ('knn', knn_model)])
eclf = eclf.fit(Z_train, y_train)
print(eclf.predict(Z_test).astype(bool))

[False False  True  True  True  True  True False  True  True  True False
 False  True  True False  True  True  True  True]
